In [7]:
import os
import json
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from src.db import DataBase
from src.utils_general import beeps
from src.utils_date import add_days
from src.utils_stocks import get_ls_sym
from src.utils_stocks import get_df_prices
from src.utils_stocks import suppress_stdout
try:
    with open('dir.txt') as f:
        dir_db = json.load(f)['dir_db']
except:
    dir_db = ''
    assert dir_db, 'Please provide a directory to save the sqlite database, in the "dir_db" variable!'

def db_remove_dups_prices_m(db, date_str):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM prices_m
         WHERE DATE(datetime)>='{}'
           AND ROWID NOT IN
               (SELECT MAX(ROWID)
                 FROM prices_m
                WHERE DATE(datetime)>='{}'
                GROUP BY sym, datetime)
    '''.format(date_str, date_str)
    db.execute(q)

def db_remove_dups_prices_d(db, date_str):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM prices_d
         WHERE DATE(date)>='{}'
           AND ROWID NOT IN
               (SELECT MAX(ROWID)
                 FROM prices_d
                WHERE DATE(date)>='{}'
                GROUP BY sym, date)
    '''.format(date_str, date_str)
    db.execute(q)

def db_remove_dups_stocks(db):
    print('Removing duplicates...')
    q = '''
        DELETE
          FROM stocks
         WHERE ROWID NOT IN
               (SELECT MAX(ROWID)
                  FROM stocks
                 GROUP BY sym)
    '''
    db.execute(q)

ls_init_str = [
    #prices_m
    '''CREATE TABLE IF NOT EXISTS prices_m(
        sym TEXT
        ,datetime TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER
        ,is_reg_hours INTEGER)''',
    
    '''CREATE INDEX IF NOT EXISTS index_prices_m_all
        ON prices_m(sym, date(datetime), is_reg_hours)''',

    '''CREATE INDEX IF NOT EXISTS index_prices_m_date
        ON prices_m(date(datetime))''',

    #prices_d
    '''CREATE TABLE IF NOT EXISTS prices_d(
        sym TEXT
        ,date TEXT
        ,open REAL
        ,high REAL
        ,low REAL
        ,adj_close REAL
        ,volume INTEGER)''',
   
    '''CREATE INDEX IF NOT EXISTS index_prices_d_date
        ON prices_d(sym, date(date))''',
    
    #stocks
    '''CREATE TABLE IF NOT EXISTS stocks(
        sym TEXT
        ,long_name TEXT
        ,sec TEXT
        ,ind TEXT
        ,quote_type TEXT
        ,fund_family TEXT
        ,summary TEXT
        ,timestamp TEXT)''',

    '''CREATE INDEX IF NOT EXISTS index_stocks
        ON stocks(sym, quote_type)''',
    
    #stocks_error
    '''CREATE TABLE IF NOT EXISTS stocks_error(
        sym TEXT)''',
    
    #proba
    '''CREATE TABLE IF NOT EXISTS proba(
        sym TEXT
        ,datetime TEXT
        ,my_index INTEGER
        ,proba REAL
        ,datetime_update TEXT)''',
]

db = DataBase(ls_init_str, dir_db)

# Update table: stocks

In [ ]:
def get_df_info(sym):
    dt_info = yf.Ticker(sym).info
    dt_info['timestamp'] = datetime.datetime.now()
    dt_info['sector'] = dt_info.get('sector')
    dt_col = {
        'symbol':'sym',
        'longName':'long_name',
        'sector':'sec',
        'industry':'ind',
        'quoteType':'quote_type',
        'fundFamily':'fund_family',
        'longBusinessSummary':'summary',
        'timestamp':'timestamp',
    }
    dt_info = {key:dt_info.get(key) for key in dt_col}
    df = pd.DataFrame([dt_info])
    df = df.rename(columns=dt_col)
    return df

#get ls_sym
ls_sym = get_ls_sym()
q = '''
    --SELECT sym FROM STOCKS UNION ALL
    SELECT sym FROM stocks_error UNION ALL
    SELECT sym FROM stocks WHERE summary IS NOT NULL
'''
ls_sym_exclude = pd.read_sql(q, db.conn)['sym'].to_list()
ls_sym = [x for x in ls_sym if x not in ls_sym_exclude]
#extract and load
ls_df = []
for i, sym in enumerate(ls_sym):
    try:
        if (i+1)%50==0 and ls_df:
            df = pd.concat(ls_df)
            print('Saving...')
            df.to_sql('stocks', db.conn, if_exists='append', index=0)
            ls_df = []
        print(sym, i+1, len(ls_sym), end=' ')
        df_info = get_df_info(sym)
        ls_df.append(df_info)
        print('Done.')
    except Exception as e:
        print(f'Skipped... ({type(e).__name__}: {str(e)})')
        df = pd.DataFrame([{'sym':sym}])
        df.to_sql('stocks_error', db.conn, if_exists='append', index=0)
#load remaining
if ls_df:
    df = pd.concat(ls_df)
    df.to_sql('stocks', db.conn, if_exists='append', index=0)
#remove duplicates
db_remove_dups_stocks(db)
beeps()

# Update table: prices_m

In [3]:
def get_ls_df_prices_m(ls_sym, ls_date_str, ls_sym_exclude=[]):
    count = 0
    count_e = 0
    ls_df = []
    for i, sym in enumerate(ls_sym):
        count+=1
        try:
            if i%100==0:
                print(i+1, len(ls_sym))
            df = get_df_prices(sym, ls_date_str[0], ls_date_str[-1])
            len_unique_dates = len(df['datetime'].dt.date.unique())
            len_candles_per_day = df.shape[0]/len_unique_dates
            if len_unique_dates<len(ls_date_str):
                print('Skip: Incomplete data', sym, len_unique_dates)
            elif len_candles_per_day<200:
                print('Skip: Not enough candles per day', sym, len_candles_per_day)
            else:
                ls_df.append(df)
        except Exception as e:
            print(f'Error: {sym}, {type(e).__name__}: {str(e)}')
            count_e+=1
    print(count, count_e, round(count_e/count, 3))
    return ls_df

# get max date present
q = '''
    SELECT DATE(MAX(datetime))
      FROM prices_m
     WHERE sym='IBM'
'''
max_date_str = pd.read_sql(q, db.conn).iloc[0,0]
# get missing dates
df = yf.download('IBM', period='1y', interval='1d', progress=0).reset_index()
df['Date']= df['Date'].astype('str')
ls_date_str = df[df['Date']>max_date_str]['Date'].to_list()
assert ls_date_str, 'No dates to extract!'
print('Extracing prices in ', ls_date_str)
# get ls_sym
q = '''
    SELECT sym
      FROM stocks
     WHERE sec IS NOT NULL
'''
ls_sym = pd.read_sql(q, db.conn)['sym'].to_list()
# extract and load
ls_df = get_ls_df_prices_m(ls_sym, ls_date_str)
if ls_df:
    df = pd.concat(ls_df)
    df.to_sql('prices_m', db.conn, if_exists='append', index=0)
    db_remove_dups_prices_m(db, ls_date_str[0])
    beeps()

AssertionError: No dates to extract!

# Update table: prices_d

In [6]:
# get max date present
q = '''
    SELECT DATE(MAX(date))
      FROM prices_d
     WHERE sym='IBM'
'''
max_date_str = pd.read_sql(q, db.conn).iloc[0,0]
# check dates
end = add_days(datetime.datetime.today().strftime('%Y-%m-%d'), 3) #today's date plus 3 days
df = yf.download('IBM', start=max_date_str, end=end, interval='1d', progress=0).reset_index()
df = df[df['Date'].astype('str')>max_date_str]
assert not df.empty, 'No dates to extract...'
print(f'Extracting from {max_date_str} plus 1, to {end}')

Extracting from 2020-12-04 plus 1, to 2020-12-15


In [8]:
# get ls_sym
q = '''
    SELECT sym
      FROM stocks
     WHERE sec IS NOT NULL
'''
ls_sym = pd.read_sql(q, db.conn)['sym'].to_list()
# extract
dt_cols = {
    'sym':'sym',
    'Date':'date',
    'Open':'open',
    'High':'high',
    'Low':'low',
    'Adj Close':'adj_close',
    'Volume':'volume',
}
count = 0
count_e = 0
ls_df = []
for i, sym in enumerate(ls_sym):
    count+=1
    try:
        if i%300==0:
            print(i+1, len(ls_sym))
        #df = yf.download(sym, start=max_date_str, end=end, interval='1d', progress=0).reset_index()
        #df = df[df['Date'].astype('str')>max_date_str]
        with suppress_stdout():
            df = yf.download(sym, period = '1mo', interval='1d', progress=0).reset_index()
        df['sym'] = sym
        df = df.rename(columns=dt_cols)
        df = df[list(dt_cols.values())]
        ls_df.append(df)
    except Exception as e:
        print(f'Error: {sym}, {type(e).__name__}: {str(e)}')
        count_e+=1
if ls_df:
    # load
    print(count, count_e, round(count_e/count, 3))
    df = pd.concat(ls_df)
    df.to_sql('prices_d', db.conn, if_exists='append', index=0)
    db_remove_dups_prices_d(db, max_date_str)
    beeps()

1 4303
301 4303
601 4303
901 4303
1201 4303
1501 4303
1801 4303
2101 4303
2401 4303
2701 4303
3001 4303
3301 4303
3601 4303
3901 4303
4201 4303
4303 0 0.0
Removing duplicates...


# Manual Editting

In [70]:
#execute
assert 0
q = '''
ALTER TABLE trading_days
RENAME COLUMN Date TO date
'''
q = '''
drop table trading_days
'''
q='''
ALTER TABLE prices_interday
  RENAME TO prices_d
'''
q='''
drop index index_prices_interday
'''

q='''
    UPDATE prices_m
       SET is_reg_hours = CASE
                           WHEN time(datetime) < time('09:30:00') THEN 0
                           WHEN time(datetime) > time('15:59:00') THEN 0
                           ELSE 1
                           END
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
q='''
    --UPDATE prices_m
       SET datetime = DATETIME(datetime, '-1 hours')
     WHERE DATE(datetime) >= '2020-11-02'
       AND DATE(datetime) <= '2020-11-24'
'''
db.execute(q)
beeps()

In [8]:
#read
q = '''
SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

'''
q='''
    select type, name, tbl_name, sql
    FROM sqlite_master
    WHERE type='index'
'''
q='''
    SELECT 
    --stocks.sym
           stocks.long_name
           ,stocks.sec
           ,stocks.ind
           ,stocks.quote_type
           ,stocks.fund_family
           ,stocks.summary
           ,prices_d.*
      FROM prices_d
      LEFT JOIN stocks
        ON stocks.sym = prices_d.sym
     WHERE DATE(date) = (SELECT MAX(DATE(date)) FROM prices_d WHERE sym='IBM')
       AND adj_close > 5
       AND volume > 100000
'''
df = pd.read_sql(q, db.conn)
#df.to_csv('temp.csv', index=0)
df.iloc[:]

,long_name,sec,ind,quote_type,fund_family,summary,sym,date,open,high,low,adj_close,volume
0,"CareDx, Inc",Healthcare,Diagnostics & Research,EQUITY,None,"CareDx, Inc., a precision medicine company, di...",CDNA,2020-12-04 00:00:00,61.939999,65.285004,60.020000,64.910004,467900
1,"MacroGenics, Inc.",Healthcare,Biotechnology,EQUITY,None,"MacroGenics, Inc., a biopharmaceutical company...",MGNX,2020-12-04 00:00:00,22.840000,24.190001,22.709999,24.000000,563700
2,"Advanced Energy Industries, Inc.",Industrials,Electrical Equipment & Parts,EQUITY,None,"Advanced Energy Industries, Inc. designs, manu...",AEIS,2020-12-04 00:00:00,98.080002,101.870003,98.080002,100.919998,444500
3,Calamos Convertible Opportunities and Income Fund,Financial Services,Asset Management,EQUITY,None,Calamos Convertible Opportunities and Income F...,CHI,2020-12-04 00:00:00,13.070000,13.280000,13.070000,13.250000,210300
4,"Exponent, Inc.",Industrials,Consulting Services,EQUITY,None,"Exponent, Inc., together with its subsidiaries...",EXPO,2020-12-04 00:00:00,81.949997,83.300003,81.949997,83.099998,193800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,"MiMedx Group, Inc.",Healthcare,Medical Devices,EQUITY,None,"MiMedx Group, Inc., a biopharmaceutical compan...",MDXG,2020-12-04 00:00:00,6.500000,6.600000,6.150000,6.190000,570300
2392,Bally's Corporation,Consumer Cyclical,Resorts & Casinos,EQUITY,None,Bally's Corporation owns and operates gaming a...,BALY,2020-12-04 00:00:00,45.200001,45.939999,43.520000,45.709999,360300
2393,"USA Technologies, Inc.",Technology,Software—Infrastructure,EQUITY,None,"USA Technologies, Inc. provides wireless netwo...",USAT,2020-12-04 00:00:00,8.770000,9.810000,8.770000,9.440000,291600
2394,"Wrap Technologies, Inc.",Technology,Scientific & Technical Instruments,EQUITY,None,"Wrap Technologies, Inc., a security technology...",WRAP,2020-12-04 00:00:00,5.350000,5.625000,5.350000,5.530000,336000


In [41]:
beeps()

In [5]:
get_df_prices('BYND', '2020-11-28', '2020-12-03')

,sym,datetime,open,high,low,adj_close,volume,is_reg_hours
133,BYND,2020-11-30 05:21:00,141.900000,141.900000,141.900000,141.900000,0,0
134,BYND,2020-11-30 07:07:00,140.360000,140.360000,140.350000,140.350000,0,0
135,BYND,2020-11-30 08:17:00,140.530000,140.530000,140.530000,140.530000,0,0
136,BYND,2020-11-30 08:18:00,140.900000,140.900000,140.900000,140.900000,0,0
137,BYND,2020-11-30 08:33:00,141.180000,141.250000,141.180000,141.250000,0,0
...,...,...,...,...,...,...,...,...
1066,BYND,2020-12-02 10:50:00,135.070007,135.388702,134.800003,135.070007,11643,1
1067,BYND,2020-12-02 10:51:00,134.860001,135.559998,134.800003,135.490005,14073,1
1068,BYND,2020-12-02 10:52:00,135.489899,135.930206,135.445007,135.930206,10986,1
1069,BYND,2020-12-02 10:53:00,135.759995,136.339996,135.759995,136.294998,14608,1


In [19]:
dt_info = yf.Ticker('CDNA').info